## Introduction
The purpose of this algorithm is to find duplicate records that have different UTC time stamps between different uploads. It does NOT consider the case where there is duplicate data within an upload.

This algorithm is also NOT designed to handle the case where duplicate data has the same time stamp, as there are more efficient functions  (e.g., the duplicated function in pandas) that can quickly find and delete duplicated data. In fact, one should get rid of those types of duplicates before running this algorithm.  

Also, there are two key steps in the de-duplication process: 1) identifying duplicates, and 2) getting rid of the bad duplicate(s). This algorithm only addresses (1); however, if we can find all cases of duplicates from step (1) it should allow us to examine and solve (2).

This method is comprehensive in the sense that it looks for duplicates between every unique pair-wise combination of uploadIds for a given user. While this may be overkill, as it may compare data that is separted by several years, it is important that we don't make any assumptions about the actual time of the data, as the whole point of the algorithm is to find duplicates where the times are different. Further, we have seen a few cases where the UTC time can be off by months or years.

## Algorithm Logic

This algorithm finds sequences of cgm values that are duplicated between two different uploadIds. Here is the basic logic of the algorithm. For one user's data, we loop through each unique pair-wise combination of uploadIds. We do the following to each uploadId:
* First we round all of the cgm data to the nearest 5 minutes.
* Second we create a contiguous time series between the first data point and the last data point in the time series.
* We then merge the two time series, so that the missing cgm points are filled with nans.

We then repeat for the following for each unique pairwise combination of uploadId:
* We take the longer/larger time series and call it TL, and the shorter one Ts, which is useful for keeping track of which indices match.
* At this step, there is an optional preprocessing step that orders the shift indices to speed up the algorithm (see details in the second read example below).
* Next we shift Ts over TL, and at each step we calculate the element by element difference between cgm values. If there is an exact match, then the difference will be zero.
* At each step we count the number of zeros, and if it exceeds an algorithm defined threshold, we tag the sequence in both time series as being duplicate. 

For the examples below we will focus on cgm time series data, but if there are other data types that are missing deviceTime data and/or have incorrect UTC times, this algorithm can be adapted to those situations too. The following examples are given below:
* a very simple (fake) illustrative example
* an example with real Tidepool donor data

In [62]:
# load in the required libraries
import os
import pandas as pd
import numpy as np
from scipy.signal import correlate
import time

## real world example
for this example we will look at 1 of the 305 unique combinations,
for one donor

In [65]:
# set the minimum number of duplicate data points that are
# required to flag the duplicates 
minThreshold = 96  # 288

# load in data
hashID = "0289cfb8bd6d61ccf1f31c07aa146b7b14f0eb74474be4311860d9d77dd30f15"
dataPath = os.path.join(".", "data")
data = pd.read_csv(os.path.join(dataPath, hashID + ".csv"), low_memory=False)

# this is a great example, becuase there are missing data points in both
# time series
uploadId_A = "upid_ff6bf4b6fde9c9bc45bb211de131d225"
uploadId_B = "upid_12164f5817e09ab7bffb439d8c260131"

In [66]:
cIndex = 0
results = pd.DataFrame(columns=["uploadId_A", "span_uploadId_A",
                                "n_uploadId_A",
                                "uploadId_B", "span_uploadId_B",
                                "n_uploadId_B", "elapsedTime",
                                "correlationOfIndex", "nDuplicates",
                                "averageTimeDifference",
                                "startIndex_A", "endIndex_A",
                                "startIndex_B", "endIndex_B"])

In [67]:
# prepare the data for the algorithm

# work with just the cgm data (though this can be modified for pump data)
cgm = data.loc[data.type == "cbg", ["deviceTime", "time", "value", "uploadId"]]

# convert mmol/L to mg/dL
cgm = cgm.rename(columns={"value": "mmol_L"})
cgm["mg_dL"] = (cgm["mmol_L"] * 18.01559).astype(int)

# round utc time to the nearest 30 seconds, and then to nearest 5 minutes
ns5min=5*60*1E9
ns30sec=0.5*60*1E9
cgm["roundedTime30sec"] = pd.to_datetime(cgm["time"]).dt.round("30S")
#    pd.to_datetime((pd.to_datetime(cgm.time).astype(np.int64) // ns30sec) * ns30sec)

cgm["roundedTime5min"] = \
pd.to_datetime((pd.to_datetime(cgm["roundedTime30sec"]).astype(np.int64) // ns5min + 1) * ns5min)

In [68]:
# grab data for just the two uploads
cgm_A = cgm[cgm.uploadId == uploadId_A].reset_index().rename(columns={"index":"originalIndex"})
cgm_B = cgm[cgm.uploadId == uploadId_B].reset_index().rename(columns={"index":"originalIndex"})

In [69]:
duration0 = pd.to_datetime(cgm[cgm.uploadId == uploadId_A].time.max()) - \
    pd.to_datetime(cgm[cgm.uploadId == uploadId_A].time.min())
duration1 = pd.to_datetime(cgm[cgm.uploadId == uploadId_B].time.max()) - \
    pd.to_datetime(cgm[cgm.uploadId == uploadId_B].time.min())

if duration0 >= duration1:
    results.loc[cIndex, ["uploadId_A"]] = uploadId_A
    results.loc[cIndex, ["span_uploadId_A"]] = duration0
    results.loc[cIndex, ["n_uploadId_A"]] = len(cgm_A)
    results.loc[cIndex, ["uploadId_B"]] = uploadId_B
    results.loc[cIndex, ["span_uploadId_B"]] = duration1
    results.loc[cIndex, ["n_uploadId_B"]] = len(cgm_B)

In [70]:
# create a continguous time series from the first to last data point
contiguousBeginDateTime_A = min(cgm_A.roundedTime5min)
contiguousEndDateTime_A = max(cgm_A.roundedTime5min)
rng_A = pd.date_range(contiguousBeginDateTime_A, contiguousEndDateTime_A, freq="5min")
contiguousData_A = pd.DataFrame(rng_A, columns=["dateTime"])

# merge data
contiguousData_A = pd.merge(contiguousData_A, cgm_A,
                            left_on="dateTime", right_on="roundedTime5min",
                            how="left")

contiguousBeginDateTime_B = min(cgm_B.roundedTime5min)
contiguousEndDateTime_B = max(cgm_B.roundedTime5min)
rng_B = pd.date_range(contiguousBeginDateTime_B, contiguousEndDateTime_B, freq="5min")
contiguousData_B = pd.DataFrame(rng_B, columns=["dateTime"])

# merge data
contiguousData_B = pd.merge(contiguousData_B, cgm_B,
                            left_on="dateTime", right_on="roundedTime5min",
                            how="left")

In [71]:
# build faster indexing with cross correlation here

# 1. figure out the median of TL and Ts combined (T_all), so that 
# data can be separated into an equal number of positive and negative
# values.

T_all = pd.concat([contiguousData_A.mg_dL, contiguousData_B.mg_dL])
median_all = T_all.median()
min_all = T_all.min()
max_all = T_all.max()

# scale TL and Ts data so that the following three things happen:
# 1. median_all <= value <= max_all gets normalized between 1 and 1.1
# 2. min_all <= value < median_all gets normalized between -1.1 and -1 
# 3. all nans get replaced with zeros
def normalize(value, valMin, valMax, normMin, normMax):
    a = (normMax - normMin)/(valMax - valMin)
    b = normMax - a * valMax
    newvalue = a * value + b
    
    return newvalue

# TODO: turn this into a function
TL1 = normalize(contiguousData_A.mg_dL[contiguousData_A.mg_dL >= median_all],
                median_all, max_all, 1, 1.1)

TL2 = normalize(contiguousData_A.mg_dL[contiguousData_A.mg_dL < median_all],
                min_all, median_all, -1.1, -1)

TL3 = contiguousData_A.mg_dL[contiguousData_A.mg_dL.isna()].fillna(0)

TL_norm = pd.concat([TL1, TL2, TL3]).sort_index() 

Ts1 = normalize(contiguousData_B.mg_dL[contiguousData_B.mg_dL >= median_all],
                median_all, max_all, 1, 1.1)

Ts2 = normalize(contiguousData_B.mg_dL[contiguousData_B.mg_dL < median_all],
                min_all, median_all, -1.1, -1)

Ts3 = contiguousData_B.mg_dL[contiguousData_B.mg_dL.isna()].fillna(0)

Ts_norm = pd.concat([Ts1, Ts2, Ts3]).sort_index() 

# do the cross correlation to get the most likely indices that contain duplicates
xCorr = pd.Series(correlate(TL_norm, Ts_norm), name="value")
indicesToTry = pd.DataFrame(xCorr[xCorr > 1])
indicesToTry["TLindex"] = indicesToTry.index + 1 - len(contiguousData_B)
indicesToTry = indicesToTry.sort_values(by="value", ascending=False)

In [72]:
TL = np.array(contiguousData_A.mg_dL)
Ts = np.array(contiguousData_B.mg_dL)
t = time.time()

for i in indicesToTry["TLindex"]:
    print("trying index", i)
    tempTL = TL[max([0, i]):min([len(TL), (len(Ts) + i)])]
    tempDiff = tempTL - Ts[-len(tempTL):]
    nZeros = sum(tempDiff == 0)
    if nZeros >= minThreshold:
        print("found it at index", i)
        results.loc[cIndex, ["correlationOfIndex"]] = round(indicesToTry[indicesToTry["TLindex"] == i].value.values[0],1)
        dupTL = contiguousData_A[max([0, i]):min([len(TL), (len(Ts) + i)])]
        dupTs = contiguousData_B[-len(tempTL):]
        combined = pd.concat([dupTL.reset_index(drop=True).add_suffix(".TL"),
                              dupTs.reset_index(drop=True).add_suffix(".Ts")], axis=1)

        results.loc[cIndex, ["nDuplicates"]] = nZeros
        results.loc[cIndex, ["startIndex_A"]] = \
            combined.loc[combined["mg_dL.TL"].notnull(), "originalIndex.TL"].min()
        results.loc[cIndex, ["endIndex_A"]] = \
            combined.loc[combined["mg_dL.TL"].notnull(), "originalIndex.TL"].max()

        results.loc[cIndex, ["startIndex_B"]] = \
            combined.loc[combined["mg_dL.Ts"].notnull(), "originalIndex.Ts"].min()
        results.loc[cIndex, ["endIndex_B"]] = \
            combined.loc[combined["mg_dL.Ts"].notnull(), "originalIndex.Ts"].max()

        cTimeDifference = pd.to_datetime(combined["time.TL"]) - \
                            pd.to_datetime(combined["time.Ts"])

        averageTimeDifference = cTimeDifference.dt.seconds.mean()
        results.loc[cIndex, ["averageTimeDifference"]] = averageTimeDifference

        break

elapsedTime = time.time() - t
results.loc[cIndex, ["elapsedTime"]] = elapsedTime
print("finished and it took", round(elapsedTime, 3), "secs")

trying index 181
found it at index 181
finished and it took 0.049 secs


In [73]:
results.T

,0
uploadId_A,upid_ff6bf4b6fde9c9bc45bb211de131d225
span_uploadId_A,30 days 21:43:35
n_uploadId_A,1000
uploadId_B,upid_12164f5817e09ab7bffb439d8c260131
span_uploadId_B,29 days 08:38:38
n_uploadId_B,288
elapsedTime,0.0487611
correlationOfIndex,326.2
nDuplicates,288
averageTimeDifference,0


In [74]:
# here are duplicated results (pan left and right to see the full DataFrame)
combined

,dateTime.TL,originalIndex.TL,deviceTime.TL,time.TL,mmol_L.TL,uploadId.TL,mg_dL.TL,roundedTime30sec.TL,roundedTime5min.TL,dateTime.Ts,originalIndex.Ts,deviceTime.Ts,time.Ts,mmol_L.Ts,uploadId.Ts,mg_dL.Ts,roundedTime30sec.Ts,roundedTime5min.Ts
0,2016-01-22 11:05:00,29090.0,2016-01-22T02:59:46,2016-01-22T11:02:23.000Z,11.490048,upid_ff6bf4b6fde9c9bc45bb211de131d225,207.0,2016-01-22 11:02:30,2016-01-22 11:05:00,2016-01-22 11:05:00,29091.0,2016-01-22T02:59:46,2016-01-22T11:02:23.000Z,11.490048,upid_12164f5817e09ab7bffb439d8c260131,207.0,2016-01-22 11:02:30,2016-01-22 11:05:00
1,2016-01-22 11:10:00,29093.0,2016-01-22T03:04:46,2016-01-22T11:07:23.000Z,10.657436,upid_ff6bf4b6fde9c9bc45bb211de131d225,191.0,2016-01-22 11:07:30,2016-01-22 11:10:00,2016-01-22 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,2016-01-22 11:15:00,29095.0,2016-01-22T03:09:46,2016-01-22T11:12:22.000Z,10.934974,upid_ff6bf4b6fde9c9bc45bb211de131d225,196.0,2016-01-22 11:12:30,2016-01-22 11:15:00,2016-01-22 11:15:00,29094.0,2016-01-22T03:09:46,2016-01-22T11:12:22.000Z,10.934974,upid_12164f5817e09ab7bffb439d8c260131,196.0,2016-01-22 11:12:30,2016-01-22 11:15:00
3,2016-01-22 11:20:00,29096.0,2016-01-22T03:14:46,2016-01-22T11:17:22.000Z,9.935839,upid_ff6bf4b6fde9c9bc45bb211de131d225,178.0,2016-01-22 11:17:30,2016-01-22 11:20:00,2016-01-22 11:20:00,29097.0,2016-01-22T03:14:46,2016-01-22T11:17:22.000Z,9.935839,upid_12164f5817e09ab7bffb439d8c260131,178.0,2016-01-22 11:17:30,2016-01-22 11:20:00
4,2016-01-22 11:25:00,29098.0,2016-01-22T03:19:46,2016-01-22T11:22:22.000Z,8.714674,upid_ff6bf4b6fde9c9bc45bb211de131d225,157.0,2016-01-22 11:22:30,2016-01-22 11:25:00,2016-01-22 11:25:00,29099.0,2016-01-22T03:19:46,2016-01-22T11:22:22.000Z,8.714674,upid_12164f5817e09ab7bffb439d8c260131,157.0,2016-01-22 11:22:30,2016-01-22 11:25:00
5,2016-01-22 11:30:00,29100.0,2016-01-22T03:24:46,2016-01-22T11:27:22.000Z,7.771047,upid_ff6bf4b6fde9c9bc45bb211de131d225,140.0,2016-01-22 11:27:30,2016-01-22 11:30:00,2016-01-22 11:30:00,29101.0,2016-01-22T03:24:46,2016-01-22T11:27:22.000Z,7.771047,upid_12164f5817e09ab7bffb439d8c260131,140.0,2016-01-22 11:27:30,2016-01-22 11:30:00
6,2016-01-22 11:35:00,29103.0,2016-01-22T03:29:46,2016-01-22T11:32:22.000Z,7.549017,upid_ff6bf4b6fde9c9bc45bb211de131d225,136.0,2016-01-22 11:32:30,2016-01-22 11:35:00,2016-01-22 11:35:00,29102.0,2016-01-22T03:29:46,2016-01-22T11:32:22.000Z,7.549017,upid_12164f5817e09ab7bffb439d8c260131,136.0,2016-01-22 11:32:30,2016-01-22 11:35:00
7,2016-01-22 11:40:00,29105.0,2016-01-22T03:34:46,2016-01-22T11:37:22.000Z,7.882062,upid_ff6bf4b6fde9c9bc45bb211de131d225,142.0,2016-01-22 11:37:30,2016-01-22 11:40:00,2016-01-22 11:40:00,29104.0,2016-01-22T03:34:46,2016-01-22T11:37:22.000Z,7.882062,upid_12164f5817e09ab7bffb439d8c260131,142.0,2016-01-22 11:37:30,2016-01-22 11:40:00
8,2016-01-22 11:45:00,29107.0,2016-01-22T03:39:46,2016-01-22T11:42:22.000Z,8.659167,upid_ff6bf4b6fde9c9bc45bb211de131d225,155.0,2016-01-22 11:42:30,2016-01-22 11:45:00,2016-01-22 11:45:00,29106.0,2016-01-22T03:39:46,2016-01-22T11:42:22.000Z,8.659167,upid_12164f5817e09ab7bffb439d8c260131,155.0,2016-01-22 11:42:30,2016-01-22 11:45:00
9,2016-01-22 11:50:00,29108.0,2016-01-22T03:44:46,2016-01-22T11:47:22.000Z,8.825689,upid_ff6bf4b6fde9c9bc45bb211de131d225,159.0,2016-01-22 11:47:30,2016-01-22 11:50:00,2016-01-22 11:50:00,29109.0,2016-01-22T03:44:46,2016-01-22T11:47:22.000Z,8.825689,upid_12164f5817e09ab7bffb439d8c260131,159.0,2016-01-22 11:47:30,2016-01-22 11:50:00
